In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from time import time
import pickle
import os
from mlxtend.plotting import plot_decision_regions

In [4]:
df1 = pd.read_csv("C:/Users/CE206-26/Desktop/크롤링구글맵스라벨링1.csv",encoding='CP949')
df2 = pd.read_csv("C:/Users/CE206-26/Desktop/구글맵스띄어쓰기삭제.csv",encoding='CP949')
df_train = df1
df_test = df2

In [5]:
df_test['라벨링'] = '0' ### test 데이터로 쓰기 위해서는 라벨링 필요 ###

In [7]:
df1['라벨링'] = df1['라벨링'].astype('str') ### 라벨이 int면 라벨이라고 인식을 못함###

In [8]:
df1['라벨링'].dtypes ### dtype('O') 여야함###

dtype('O')

In [9]:
twitter = Twitter()
def tokenizer_twitter_morphs(doc):
    return twitter.morphs(doc)

def tokenizer_twitter_noun(doc):
    return twitter.nouns(doc)

def tokenizer_twitter_pos(doc):
    return twitter.pos(doc, norm=True, stem=True)

C:\Users\CE206-26\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\CE206-26\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [10]:
komoran = Komoran()
def tokenizer_noun(doc):
    return komoran.nouns(doc)

def tokenizer_morphs(doc):
    return komoran.morphs(doc)

In [11]:
### 리뷰들 토큰화 시간 오래걸림 ###

df_train['token_review'] = df_train['리뷰'].apply(tokenizer_twitter_morphs)
df_test['X_test_tokkened'] = df_test['리뷰'].apply(tokenizer_twitter_morphs)
tokens = [ t for d in df_train['token_review'] for t in d]

In [12]:
import nltk
text = nltk.Text(tokens, name='NMSC')

In [14]:
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, c) for d, c in df_train[['token_review', '라벨링']].values]
tagged_test_docs = [TaggedDocument(d, c) for d, c in df_test[['X_test_tokkened','라벨링']].values]

In [17]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

4

In [18]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [19]:
from gensim.models import Doc2Vec

2019-08-22 09:38:02,796 : INFO : 'pattern' package not found; tag filters are not available for English


In [20]:
doc_vectorizer = Doc2Vec()

In [21]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

2019-08-22 09:38:10,223 : INFO : collecting all words and their counts
2019-08-22 09:38:10,224 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-08-22 09:38:10,325 : INFO : collected 31035 word types and 6 unique tags from a corpus of 487 examples and 385571 words
2019-08-22 09:38:10,326 : INFO : Loading a fresh vocabulary
2019-08-22 09:38:10,355 : INFO : effective_min_count=5 retains 6669 unique words (21% of original 31035, drops 24366)
2019-08-22 09:38:10,356 : INFO : effective_min_count=5 leaves 348256 word corpus (90% of original 385571, drops 37315)
2019-08-22 09:38:10,384 : INFO : deleting the raw counts dictionary of 31035 items
2019-08-22 09:38:10,386 : INFO : sample=0.001 downsamples 41 most-common words
2019-08-22 09:38:10,387 : INFO : downsampling leaves estimated 278339 word corpus (79.9% of prior 348256)
2019-08-22 09:38:10,416 : INFO : estimated required memory for 6669 words and 100 dimensions: 8673300 bytes
2019-08-22 09:38:10,418 : I

Doc2Vec(dm/m,d100,n5,w5,mc5,s0.001,t3)


In [22]:
doc_vectorizer.corpus_count

487

In [23]:
doc_vectorizer.iter

C:\Users\CE206-26\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [24]:
start = time()
for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time()
print("During Time: {}".format(end-start))

C:\Users\CE206-26\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until
2019-08-22 09:38:23,826 : INFO : training model with 3 workers on 6669 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-08-22 09:38:24,151 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 09:38:24,162 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-22 09:38:24,166 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-22 09:38:24,167 : INFO : EPOCH - 1 : training on 385571 raw words (278383 effective words) took 0.3s, 828070 effective words/s
2019-08-22 09:38:24,473 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 09:38:24,486 : INFO : worker thread finished; awaiting finish of 1 more threads

2019-08-22 09:38:29,661 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 09:38:29,668 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-22 09:38:29,670 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-22 09:38:29,672 : INFO : EPOCH - 3 : training on 385571 raw words (279067 effective words) took 0.3s, 926505 effective words/s
2019-08-22 09:38:29,964 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 09:38:29,967 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-22 09:38:29,980 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-22 09:38:29,981 : INFO : EPOCH - 4 : training on 385571 raw words (278866 effective words) took 0.3s, 915424 effective words/s
2019-08-22 09:38:30,286 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 09:38:30,296 : INFO : worker thread finished; awaiting finish of 1 more threads
20

2019-08-22 09:38:35,124 : INFO : training model with 3 workers on 6669 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-08-22 09:38:35,434 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 09:38:35,441 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-22 09:38:35,449 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-22 09:38:35,450 : INFO : EPOCH - 1 : training on 385571 raw words (278646 effective words) took 0.3s, 874729 effective words/s
2019-08-22 09:38:35,740 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-22 09:38:35,745 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-22 09:38:35,749 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-22 09:38:35,750 : INFO : EPOCH - 2 : training on 385571 raw words (278966 effective words) took 0.3s, 942100 effective words/s
2019-08-22 09:38:36,106 : INFO : wo

During Time: 16.621432304382324


In [25]:
model_name = 'Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model'
doc_vectorizer.save(model_name)

2019-08-22 09:40:35,927 : INFO : saving Doc2Vec object under Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model, separately None
2019-08-22 09:40:36,013 : INFO : saved Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model


In [94]:
doc_vectorizer = Doc2Vec.load(model_name)

2019-08-22 11:00:14,392 : INFO : loading Doc2Vec object from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model
2019-08-22 11:00:14,462 : INFO : loading vocabulary recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.vocabulary.* with mmap=None
2019-08-22 11:00:14,464 : INFO : loading trainables recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.trainables.* with mmap=None
2019-08-22 11:00:14,466 : INFO : loading wv recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.wv.* with mmap=None
2019-08-22 11:00:14,467 : INFO : loading docvecs recursively from Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model.docvecs.* with mmap=None
2019-08-22 11:00:14,469 : INFO : loaded Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model


In [27]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags for doc in tagged_train_docs]

In [28]:
X_test = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]

In [29]:
y_test = [doc.tags for doc in tagged_test_docs]

In [30]:
print(len(X_train), len(y_train))

487 487


In [31]:
X_train_np = np.asarray(X_train)
X_test_np = np.array(X_test)

In [32]:
y_train_np = np.asarray(y_train)
y_test_np = np.array(y_test)

In [34]:
y_train_np = np.eye(7)[y_train_np.astype('int').reshape(-1)]
y_test_np = np.eye(7)[y_test_np.astype('int').reshape(-1)]

In [44]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit()

In [75]:
from sklearn.metrics.pairwise import cosine_similarity

In [84]:
X_train[0]

array([ 0.48203564, -0.6243266 ,  0.15739882, -0.76039356, -0.57809967,
       -1.0096    ,  0.59068424,  1.2903953 , -0.1569971 , -0.49474514,
        0.59646636, -1.1688079 ,  0.7644317 ,  0.46162194,  0.56518203,
        0.10650347,  0.2075075 ,  1.1108743 ,  0.5377022 ,  0.9643238 ,
       -0.3602644 , -0.5512547 ,  0.18889445,  0.5664211 ,  0.27161875,
       -0.9321773 , -0.551525  ,  0.22762342, -0.30506584, -0.93813175,
       -0.62588614,  1.3426479 , -0.08452383,  0.3199109 , -0.15890029,
        0.90663594,  0.13211389, -0.5592338 , -0.39105272, -0.44546893,
        0.5469872 ,  0.8939225 ,  1.6384526 ,  0.18271182, -0.04576356,
        0.39523944,  1.1815152 , -0.46333578, -1.1842191 , -0.5022238 ,
        0.6218391 ,  0.659987  ,  0.3665604 ,  0.7438687 ,  0.12923557,
        0.06497975, -0.61321247,  0.58036715, -0.1283809 , -0.3766289 ,
       -0.61781293, -0.31019232,  0.3512921 ,  0.08782116, -0.62938935,
        0.5141684 ,  0.4477248 ,  0.97339994, -0.12809664,  0.55

In [35]:
from sklearn.neighbors import KNeighborsClassifier

In [36]:
ml = KNeighborsClassifier(n_neighbors = 6, p=2, metric='minkowski')

In [46]:
ml.fit(X_train_np,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')


In [38]:
y_pred = ml.predict(X_test)

In [103]:
X_pred = ml.predict(X_train)

In [106]:
y_test

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0'

In [183]:
print('총 테스트 개수:%d, 오류개수:%d' %(len(y_test), (y_test != y_pred).sum()))
print('정확도: %.3f' %accuracy_score(y_test,y_pred))

총 테스트 개수:974, 오류개수:974
정확도: 0.000


In [39]:
from sklearn.metrics import accuracy_score

In [40]:
print('총 테스트 개수:%d, 오류개수:%d' %(len(y_test), (y_test != y_pred).sum()))
print('정확도: %.3f' %accuracy_score(y_test,y_pred))

총 테스트 개수:974, 오류개수:974
정확도: 0.000


In [43]:
print(y_pred)

['6' '4' '2' '1' '4' '1' '1' '1' '1' '6' '4' '6' '1' '6' '6' '1' '1' '1'
 '4' '6' '2' '1' '2' '6' '4' '4' '1' '6' '6' '4' '1' '4' '4' '2' '4' '1'
 '4' '1' '1' '1' '4' '4' '6' '1' '1' '4' '1' '4' '4' '4' '2' '2' '4' '4'
 '6' '1' '4' '1' '6' '6' '4' '1' '1' '3' '6' '6' '4' '1' '3' '6' '6' '1'
 '1' '6' '1' '3' '1' '1' '4' '1' '1' '1' '1' '6' '1' '1' '4' '1' '1' '4'
 '1' '1' '3' '1' '6' '1' '2' '6' '1' '4' '1' '6' '1' '1' '2' '4' '1' '6'
 '2' '1' '4' '4' '1' '4' '1' '6' '4' '6' '2' '4' '1' '6' '4' '4' '1' '6'
 '2' '1' '1' '4' '4' '1' '6' '6' '4' '1' '6' '4' '1' '4' '6' '3' '1' '6'
 '1' '1' '4' '1' '1' '1' '1' '4' '1' '6' '1' '1' '6' '1' '4' '6' '2' '4'
 '4' '4' '1' '6' '6' '1' '4' '1' '4' '1' '4' '1' '1' '1' '1' '6' '4' '1'
 '4' '1' '1' '2' '4' '6' '6' '1' '5' '4' '6' '4' '1' '4' '6' '2' '3' '1'
 '1' '6' '2' '6' '3' '4' '6' '3' '6' '6' '1' '4' '6' '1' '4' '6' '6' '6'
 '1' '4' '1' '1' '1' '2' '2' '6' '1' '4' '3' '1' '4' '2' '6' '1' '1' '1'
 '2' '6' '1' '4' '1' '1' '4' '1' '2' '4' '6' '1' '6